In [ ]:
# Install necessary libraries
!pip install firebase-admin google-cloud-firestore

# Import required libraries
import os
import textwrap
from IPython.display import Markdown
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db as firebase_db
from google.cloud import firestore
import google.generativeai as genai
from google.colab import userdata

# Set the path to your service account key JSON file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/sbhrccit003.json"

# Initialize Firebase Admin SDK
cred = credentials.Certificate("/content/sbhrccit003.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://sbhrccit003-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

# Function to convert text to Markdown format
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Set up API key
GOOGLE_API_KEY = userdata.get('GoogleAI')
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize the GenerativeModel with gemini-pro
model = genai.GenerativeModel('gemini-pro')


In [ ]:
# Initialize Firestore client
db_firestore = firestore.Client()

# Reference to the Water_Use_Cases collection in Firestore
water_use_cases_ref = db_firestore.collection('Water_Use_Cases')

# Query documents from Firestore
try:
    docs = water_use_cases_ref.get()

    # Print document data
    for doc in docs:
        print(f'{doc.id} => {doc.to_dict()}')
except Exception as e:
    print("An error occurred:", e)

In [ ]:
# Initialize an empty dictionary to store the use cases data
use_cases_data = {}

# Query documents from Firestore
try:
    docs = water_use_cases_ref.get()

    # Iterate over the documents
    for doc in docs:
        # Extract the document ID and data
        doc_id = doc.id
        doc_data = doc.to_dict()

        # Store the data in the use_cases_data dictionary
        use_cases_data[doc_id] = doc_data

except Exception as e:
    print("An error occurred:", e)

# Print the use cases data
print("Use Cases Data:")
for use_case, parameters in use_cases_data.items():
    print(f"{use_case}:")
    for parameter, value_range in parameters.items():
        print(f"  {parameter}: {value_range}")


In [ ]:
from firebase_admin import db

# Reference to the root of your Firebase Realtime Database
root_ref = db.reference("/")
realtime_data = root_ref.get()

# Assuming the realtime_data is a list with one dictionary inside, extract the dictionary
if isinstance(realtime_data, list) and len(realtime_data) > 0:
    realtime_data = realtime_data[0]

# Function to flatten nested dictionary
def flatten(dictionary, parent_key='', sep='_'):
    flat_dict = {}
    for key, value in dictionary.items():
        new_key = parent_key + sep + key if parent_key else key
        if isinstance(value, dict):
            flat_dict.update(flatten(value, new_key, sep=sep))
        else:
            flat_dict[new_key] = value
    return flat_dict

# Flatten the real-time data
flattened_realtime_data = flatten(realtime_data)

# Print the flattened dictionary
print("Flattened Real-time Data:")
for key, value in flattened_realtime_data.items():
    print(f"{key}: {value}")


In [ ]:
prompt = "[I have use cases threshold for water'\n\n[" \
         "\n\nUse Cases Data:\n\n" \
         "  " + "\n  ".join([f"{use_case}:\n    {parameter}: {value_range}"
                              for use_case, parameters in use_cases_data.items()
                              for parameter, value_range in parameters.items()]) + \
         "\n\nMy realtime database detects test water sample as\n\n[" \
         "\n\nFlattened Real-time Data:\n\n" \
         "  " + "\n  ".join([f"{key}: {value}" for key, value in flattened_realtime_data.items()]) + \
         '''
         Here the sensors are connected to a 12 Bit ADC with 0 value at 0 volts from the module and 4095 value at 3.3 Volts.

For reference in our calibration test, TDS_ADC and TDS_ppm  at distilled water shows 0 value in ADC and 0 ppm which means distilled water doesn't have any dissolved particles.
Thus 4095 ADC value represents maximum TDS which is 1000 ppm that our sensor is capable of reading. So using the  Flattened Real-time Data, calibrate the equivalent TDS value of the test sample in ppm.


Turbidity sensor has average 100% at 0 NTU, 80.50% at 500 NTU, 64.50% at 1000ntu, 45.20% at 2000NTU and 22.30% at 4000NTU. In our calibration test, Turbidity_ADC value 0 means 100% and value 4095 means 0%. So using the  Flattened Real-time Data, calibrate the equivalent turbidity of the test sample in NTU.

For reference in our calibration test, pH_ADC  of distilled water shows 2975 value in AD. Take this as reference, any value above 3020 is basic and below is acidic. Map it accordingly before classification.

State where the test samples can be used based on the use cases threshold giving priority to the use case that satisfies all the threshold values and so forth.

For example, the test sample satisfies all the threshold values of Drinking_Water and also all but one parameter’s threshold values for Facial_Water, state Drinking_Water as the best suitable use followed by  Facial_Water and so forth. And state how the sample can be modified for the best suitable use case.

Based on the Flattened Real-time Data, state the quality of the water sample and also categorise it based on the threshold value of the use cases.

Example of Final Output [Water Sample Analysis and Use Case Prioritization:

Based on the provided data, the water sample demonstrates fair quality. It meets several parameters across various use cases but falls short in turbidity for most applications.

Here's a prioritised list of potential uses for the water sample, considering full compliance with thresholds first:
 This use case aligns perfectly with all the parameters of the water sample (DO, pH, TDS, and turbidity). No modifications are needed.
 The sample meets all parameters except for turbidity, which is slightly high (4 NTU compared to the maximum of 5 NTU). Consider implementing methods like sedimentation or filtration to reduce turbidity for optimal plant health and growth.
Similar to agricultural use, turbidity is slightly above the ideal range. Apply similar turbidity reduction methods as mentioned above to ensure a healthy environment for fish.
 The water sample is suitable for gardening without any modifications.

 While within the acceptable range, the turbidity is on the higher end. Consider reducing it for optimal livestock health through filtration or sedimentation.

The TDS (780 mg/L) is significantly higher than the acceptable range (400-450 mg/L). Employ a reverse osmosis system to lower TDS levels before using it in aquariums.
 Turbidity exceeds the safe limit for drinking water. Implement filtration or other treatment methods to meet drinking water standards.
 Both pH and TDS levels are too high for facial use. Consider reverse osmosis for TDS reduction and pH adjustment methods to make it suitable.
 The TDS is too high for most hydroponic systems. Consider dilution with purer water or using a reverse osmosis system to achieve suitable TDS levels.
 The turbidity slightly exceeds the limit for laboratory-grade water. Filtration is necessary to meet the required standards.
 Turbidity is above the recommended limit for safe recreational use. Treatment to reduce turbidity is crucial.
 The sample falls within the range of what a wastewater treatment plant would handle, but it wouldn't be directly used there.

 Industrial Water, Agricultural Water, Fish Hatchery and Aquaculture, Gardening Water, Livestock Water (with slight turbidity reduction).
 Aquarium Water, Drinking Water, Facial Water, Hydroponic System, Laboratory and Research, Recreational Water.
 Wastewater Treatment Plant.

         '''

# print(prompt)

response = model.generate_content(prompt)

# Display the response in Markdown format
to_markdown(response.text)
